# Running a Power Flow

We load the simple example network from the create_network tutorial from the pandapower.networks module:

In [35]:
import pandapower as pp
import pandapower.networks

net = pandapower.networks.example_simple()
net

This pandapower network includes the following parameter tables:
   - load (1 elements)
   - sgen (1 elements)
   - gen (1 elements)
   - trafo (1 elements)
   - switch (8 elements)
   - shunt (1 elements)
   - bus (8 elements)
   - line (4 elements)
   - ext_grid (1 elements)

## Run a Power Flow and Access Results

Runing a loadflow adds seperate result table with the prefix 'res_':

In [21]:
pp.runpp(net)

In [22]:
net

This pandapower network includes the following parameter tables:
   - load (1 elements)
   - sgen (1 elements)
   - gen (1 elements)
   - trafo (1 elements)
   - switch (8 elements)
   - shunt (1 elements)
   - bus (8 elements)
   - line (4 elements)
   - ext_grid (1 elements)
 and the following results tables:
   - res_bus (8 elements)
   - res_trafo (1 elements)
   - res_ext_grid (1 elements)
   - res_line (4 elements)
   - res_shunt (1 elements)
   - res_gen (1 elements)
   - res_sgen (1 elements)
   - res_load (1 elements)

These results tables are pandas datafarmes with the same index as the element table. For example, the bus table contains all bus voltages and summed bus power injections:

In [23]:
 net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.020000,0.000000,6707.892587,7263.132453
1,1.020841,0.031878,0.000000,0.000000
2,1.020841,0.031878,0.000000,-1000.432572
3,1.024509,1.678842,0.000000,0.000000
4,1.024509,1.678842,0.000000,0.000000
5,1.030000,1.743203,-6000.000000,-3524.431825
6,1.032964,2.171348,-2000.000000,500.000000
7,1.022004,1.730935,1200.000000,2400.000000


We can now use pandas functionality to analyse the loadflow results, for example to get the minimum voltage in the medium voltage level:

In [24]:
net.res_bus[net.bus.vn_kv==20.].vm_pu.min()

1.0220036262008747

or the maxium voltage at a bus with load or generation:

In [25]:
load_or_generation_buses = set(net.load.bus.values) | set(net.sgen.bus.values) | set(net.gen.bus.values)
net.res_bus.vm_pu.loc[load_or_generation_buses].max()

1.0329642074629148

For more on how to use pandas for data analysis in pandapower, see the data_analysis tutorial

## Result tables

Each element (except the switch) has its own result table with results tailored to the specific element. Here, we just show each table. For parameters definitions, see the documentation of the datastructure.

In [26]:
net.res_bus

,vm_pu,va_degree,p_kw,q_kvar
0,1.020000,0.000000,6707.892587,7263.132453
1,1.020841,0.031878,0.000000,0.000000
2,1.020841,0.031878,0.000000,-1000.432572
3,1.024509,1.678842,0.000000,0.000000
4,1.024509,1.678842,0.000000,0.000000
5,1.030000,1.743203,-6000.000000,-3524.431825
6,1.032964,2.171348,-2000.000000,500.000000
7,1.022004,1.730935,1200.000000,2400.000000


In [27]:
net.res_ext_grid

,p_kw,q_kvar
0,6707.892587,7263.132453


In [28]:
net.res_line

,p_from_kw,q_from_kvar,p_to_kw,q_to_kvar,pl_kw,ql_kvar,i_ka,loading_percent
0,-6707.892587,-7263.132466,6710.966423,1570.752555,3.073836,-5692.379911,0.050875,8.652172
1,-5971.993164,-3579.346312,6000.000002,3524.431964,28.006838,-54.914349,0.196182,46.599019
2,-1979.312290,508.536136,2000.000000,-500.000000,20.687710,8.536136,0.057613,27.434641
3,1205.083190,2304.668151,-1200.000000,-2400.000000,5.083190,-95.331849,0.075792,18.002843


In [29]:
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,-6710.966424,-570.319979,6746.222267,766.14203,35.255843,195.822052,0.034629,0.19131,26.508646


In [30]:
net.res_load

,p_kw,q_kvar
0,1200.0,2400.0


In [31]:
net.res_sgen

,p_kw,q_kvar
0,-2000.0,500.0


In [32]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-6000.0,-3524.431825,1.743203,1.03


In [33]:
net.res_shunt

,p_kw,q_kvar,vm_pu
0,0.0,-1000.432572,1.020841


## Voltage Angles and Initialization

Maybe you wondered why even though there is a voltage angle of 50 degrees defined for the external grid: 

In [34]:
net.ext_grid.va_degree

0    50.0
Name: va_degree, dtype: float64

and a shift of 150° over the HV/MV transformer:

In [10]:
net.trafo.shift_degree

0    150.0
Name: shift_degree, dtype: float64

the voltage angles are all close to zero:

In [12]:
pp.runpp(net)
net.res_bus.va_degree

0    0.000000
1    0.031878
2    0.031878
3    1.678842
4    1.678842
5    1.743203
6    2.171348
7    1.730935
Name: va_degree, dtype: float64

That is because the standard parameter for calculate_voltage_angles is False, which means voltage angles at external grids and transformer shifts are ignored by default. In a radial network, the absolute voltage angle shifts do not have an influence on the power flow, which is why they are disabled by default. In meshed networks however, where multiple external grids are galvanically coupled, it is always necessary to calculate the voltage angles.

Suppose we want to calculate the correct voltage angles and set calculate_voltage_angles to True:

In [13]:
pp.runpp(net, calculate_voltage_angles=True)

LoadflowNotConverged: Loadflow did not converge!

Now the power flow does not converge. This can happen with large angle shifts. The solution is to use a initialization with a DC loadflow instead of a flat start, which is default behaviour:

In [14]:
pp.runpp(net, calculate_voltage_angles=True, init="dc")

Now, we can see that all voltage angles are correctly calculated:

net.res_bus.va_degree

If we already have a solution, we can also initialize the loadflow with the voltage values from the last loadflow:

In [78]:
pp.runpp(net, calculate_voltage_angles=True, init="results")
net.res_bus.va_degree

0    50.000000
1    50.031878
2    50.031878
3   -98.321158
4   -98.321158
5   -98.256797
6   -97.828652
7   -98.269065
Name: va_degree, dtype: float64

The power flow converges and yields correct results where a flat start power flow would have failed.

Initializing with previous results can save convergence time in cases where multiple power flows with simliar input parameters are carried out consecutively, such as in quasi-static time series simulations.

## Transformer Model

The parameter "trafo_model" can be used to switch between a 'pi' and a 't' transformer model:

In [129]:
pp.runpp(net, trafo_model="t")
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,-6710.966424,-570.319979,6746.222267,766.14203,35.255843,195.822052,0.034629,0.19131,26.508646


In [130]:
pp.runpp(net, trafo_model="pi")
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,-6710.960859,-570.132816,6746.223053,765.95233,35.262194,195.819514,0.034629,0.191309,26.508563


For a definition of the different transformer model see the power flow model documentation of the transformer element.

## Transformer Loading

The transformer loading can either be calculated in relation to the rated current:

In [133]:
pp.runpp(net, trafo_loading="current")
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,-6710.966424,-570.319979,6746.222267,766.14203,35.255843,195.822052,0.034629,0.19131,26.508646


or to the rated power of the transformer:

In [132]:
pp.runpp(net, trafo_loading="power")
net.res_trafo

,p_hv_kw,q_hv_kvar,p_lv_kw,q_lv_kvar,pl_kw,ql_kvar,i_hv_ka,i_lv_ka,loading_percent
0,-6710.966424,-570.319979,6746.222267,766.14203,35.255843,195.822052,0.034629,0.19131,27.158347


The transformer loading does not have an influence on other power flow results besides the loading_percent parameter.

## Generator Reactive Power Limits

The generator has reactive power limits of -3000...3000 kvar:

In [109]:
net.gen

,name,bus,p_kw,vm_pu,sn_kva,min_q_kvar,max_q_kvar,scaling,in_service,type
0,generator,5,-6000.0,1.03,NaN,-3000,3000,1.0,True,None


which are however exceeded in the power flow results, because the enforce_q_lims option defaults to False:

In [110]:
pp.runpp(net)
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-6000.0,-3524.431825,1.743203,1.03


If the enforce_q_lims parameter is set to True, the reactive power limit is complied with, while the voltage deviates from the voltage set point of the generator:

In [111]:
pp.runpp(net, enforce_q_lims=True)
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-6000.0,-3000.0,1.769989,1.027378


If you want to know what to do when a power flow does not converge, continue with the [diagnostic tutorial](diagnostic.ipynb).